In [ ]:
import os

src_real = "C:/Users/apostolia/dev/MICCAI2024_Pre_to_Any_IDC/data/ldm_data_all_phases_w_mask_cropped/ldm_data_all_phases_w_mask_cropped/test"
post_path_synth = "C:/Users/apostolia/dev/MICCAI2024_Pre_to_Any_IDC/data/gs0_cgs1p0_200steps/gs0_cgs1p0_200steps/PIL"
pre_path_real = os.path.join(src_real, "test_A")
post_mask_path_real = os.path.join(src_real, "test_B")

files, cases = [os.path.splitext(i)[0] for i in os.listdir(post_mask_path_real)], []
for f in files:
    f_splits = f.rsplit("_", 3)
    cases.append(f_splits[1])

cases = sorted(set(cases))

In [ ]:
from PIL import Image
import numpy as np 
import pandas as pd
from sklearn.metrics import mean_squared_error

width, height = 224, 224
data_intensity, data_mse = [], []
for c in cases: 
    ph_slices = [ps for ps in files if c in ps and "slice" in ps]
    phases = ["0000"] + sorted(set([(i.rsplit("_", 2)[1]) for i in  ph_slices]))
    slices = sorted(set([i.rsplit("_", 2)[2] for i in  ph_slices]))
    
    for ph in phases: 
        phase_slices_real_mean, phase_slices_synth_mean, phase_slices_real_std, phase_slices_synth_std, phase_slices_real_norm_mean, phase_slices_synth_norm_mean  = [], [], [], [], [], []
        phase_slices_mse_real_post_synth_post, phase_slices_mse_real_post_real_pre = [], []
        for s in slices:
            mask = Image.open(os.path.join(post_mask_path_real, f"Breast_MRI_{c}_0001_{s.replace('slice', 'mask')}.png"))
            if np.array(mask).size and np.count_nonzero(np.array(mask)):
                resized_mask = mask.resize((width, height), resample=Image.LANCZOS)
                if ph == "0000":
                    slice_pre_real = Image.open(os.path.join(pre_path_real, f"Breast_MRI_{c}_{ph}_{s}.png"))
                    resized_slice_pre_real = slice_pre_real.resize((width, height), resample=Image.LANCZOS)
                    
                    slice_pre_real_tumor = np.array(resized_slice_pre_real)[np.array(resized_mask) != 0]
                    slice_pre_real_non_tumor = np.array(resized_slice_pre_real)[np.array(resized_mask) == 0]
                    
                    phase_slices_real_mean.append(np.mean(slice_pre_real_tumor))
                    phase_slices_real_std.append(np.std(slice_pre_real_tumor))
                    phase_slices_real_norm_mean.append(np.mean(slice_pre_real_tumor)/np.mean(slice_pre_real_non_tumor))
                else:
                    slice_post_real = Image.open(os.path.join(post_mask_path_real, f"Breast_MRI_{c}_{ph}_{s}.png"))
                    slice_post_synth = Image.open(os.path.join(post_path_synth, f"Breast_MRI_{c}_{ph}_{s}_synthetic.png")).convert('L')
                    
                    resized_slice_post_real = slice_post_real.resize(slice_post_synth.size, resample=Image.LANCZOS)
                    
                    slice_post_real_tumor = np.array(resized_slice_post_real)[np.array(resized_mask) != 0]
                    slice_post_synth_tumor = np.array(slice_post_synth)[np.array(resized_mask) != 0]
                    slice_post_real_non_tumor = np.array(resized_slice_post_real)[np.array(resized_mask) == 0]
                    slice_post_synth_non_tumor = np.array(slice_post_synth)[np.array(resized_mask) == 0]
                    
                    phase_slices_real_mean.append(np.mean(slice_post_real_tumor))
                    phase_slices_synth_mean.append(np.mean(slice_post_synth_tumor))
                    phase_slices_real_std.append(np.std(slice_post_real_tumor))
                    phase_slices_synth_std.append(np.std(slice_post_synth_tumor))
                    phase_slices_real_norm_mean.append(np.mean(slice_post_real_tumor)/np.mean(slice_post_real_non_tumor))
                    phase_slices_synth_norm_mean.append(np.mean(slice_post_synth_tumor)/np.mean(slice_post_synth_non_tumor))

                    # MSE
                    phase_slices_mse_real_post_synth_post.append(mean_squared_error(slice_post_real_tumor, slice_post_synth_tumor))

                    slice_pre_real = Image.open(os.path.join(pre_path_real, f"Breast_MRI_{c}_0000_{s}.png"))
                    resized_slice_pre_real = slice_pre_real.resize(slice_post_synth.size, resample=Image.LANCZOS)
                    slice_pre_real_tumor = np.array(resized_slice_pre_real)[np.nonzero(np.array(resized_mask))]
                    phase_slices_mse_real_post_real_pre.append(mean_squared_error(slice_post_real_tumor, slice_pre_real_tumor))

        if phase_slices_real_mean:   
            phase = "Pre-contrast" if ph == "0000" else int(ph)
            data_intensity.append([c, phase, np.mean(phase_slices_real_mean), np.mean(phase_slices_real_std), np.mean(phase_slices_real_norm_mean), np.std(phase_slices_real_norm_mean), "Real"])
            if ph != "0000":
                data_intensity.append([c, int(ph), np.mean(phase_slices_synth_mean), np.mean(phase_slices_synth_std), np.mean(phase_slices_synth_norm_mean), np.std(phase_slices_synth_norm_mean), "Synthetic"])
                
                data_mse.append([c, int(ph), np.mean(phase_slices_mse_real_post_synth_post), np.std(phase_slices_mse_real_post_synth_post), "Real post-contr. vs. Synthetic post-contr."])
                data_mse.append([c, int(ph), np.mean(phase_slices_mse_real_post_real_pre), np.std(phase_slices_mse_real_post_real_pre), "Real post-contr. vs. Real pre-contr."])

df_intensity = pd.DataFrame(data_intensity, columns=["Breast_MRI", "Phase", "Mean pixel intensity", "Standard deviation of mean", "Normalized mean pixel intensity", "Standard deviation", "Type"])
df_mse = pd.DataFrame(data_mse, columns=["Breast_MRI", "Phase", "Mean MSE", "Standard deviation", "Type"])

Intensity plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.scatterplot(data=df_intensity, x="Breast_MRI", y="Normalized mean pixel intensity", style="Type", hue="Phase", size="Standard deviation", sizes=(20, 200), palette=palette, legend="brief") 
for case, i in zip(df_intensity.Breast_MRI.unique().tolist(), range(len(df_intensity.Breast_MRI.unique().tolist()))):
    plt.plot([i, i], [df_intensity[(df_intensity.Breast_MRI==case) & (df_intensity.Phase==1) & (df_intensity.Type=="Real")]["Normalized mean pixel intensity"].values[0], df_intensity[(df_intensity.Breast_MRI==case) & (df_intensity.Phase==1) & (df_intensity.Type=="Synthetic")]["Normalized mean pixel intensity"].values[0]], 
             linewidth=1, color=palette[1], zorder=0)
    
handles, lables = ax.get_legend_handles_labels()
for h in handles[0:6]+handles[12:]:
    h.set_markersize(10)
lgnd = plt.legend(handles, lables, bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel('Test case', fontsize=12)
plt.ylabel('Normalized mean pixel intensity', fontsize=12)
plt.show()
# plt.savefig('plots/20240306_gs0_cgs1p0_200steps_scatter_real_synthetic_NORMALIZED_INTENSITY_all_phases_downsize_real_mask.png', bbox_inches='tight', dpi=300)

MSE plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.scatterplot(data=df_mse, x="Breast_MRI", y="Mean MSE", style="Type", hue="Phase", size="Standard deviation", sizes=(20, 200), palette=palette, legend="brief") 
    
handles, lables = ax.get_legend_handles_labels()
for h in handles[0:6]+handles[12:]:
    h.set_markersize(10)
lgnd = plt.legend(handles, lables, bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel('Test case', fontsize=12)
plt.ylabel('Mean Squared Error (MSE) in the tumor area', fontsize=12)
plt.show()
# plt.savefig('plots/gs0_cgs1p0_200steps_scatter_MSE_all_phases_downsize_real_mask.png', bbox_inches='tight', dpi=300)

A plot with y-axis = mean intensity value, x-axis = phase in range [1,4] and visualized are for each phase accumulated over cases:  
a) real postcontrast mean intensity value as point (circle size=stdev)  
b) syn postcontrast mean intensity value as point (circle size=stdev)  
c) real pre-contrast mean intensity value (circle size = stdev).  

The circles from (c) will be in a straight line below the other two curves from (a) and (b).

In [ ]:
df_intensity_all = df_intensity.drop(columns=["Breast_MRI", "Mean pixel intensity", "Standard deviation of mean"])

df_a = df_intensity_all[(df_intensity_all.Phase != "Pre-contrast") & (df_intensity_all.Type == "Real")].drop(columns=["Type"])
a = df_a.groupby(by="Phase").mean()
if "Phase" not in a.columns:
    a["Phase"] = a.index.values
    a["Type"] = a.shape[0]*["Real"]

df_b = df_intensity_all[(df_intensity_all.Phase != "Pre-contrast") & (df_intensity_all.Type == "Synthetic")].drop(columns=["Type"])
b = df_b.groupby(by="Phase").mean()
if "Phase" not in b.columns:
    b["Phase"] = b.index.values
    b["Type"] = b.shape[0]*["Synthetic"]

df = pd.concat([a, b], ignore_index=True)

df_pre = df_intensity_all[(df_intensity_all.Phase == "Pre-contrast")].drop(columns=["Type"]).groupby(by="Phase").mean()
df_pre_ = pd.DataFrame(np.repeat(df_pre.values, 4, axis=0))
df_pre_.columns = df_pre.columns
df_pre_["Phase"] = list(range(1, 5))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(8, 4))

ax = sns.scatterplot(data=df, x="Phase", y="Normalized mean pixel intensity", hue= "Phase", style="Type", size="Standard deviation", sizes=(20, 200), legend="brief")
ax = sns.scatterplot(data=df_pre_, x="Phase", y="Normalized mean pixel intensity", color = "0.55", size="Standard deviation", sizes=(20, 200), legend=False)
ax.annotate("Pre-contrast", (3.6, 4.1), color="0.55")

for i in range(3):
    plt.plot([i+1, i+2], [df[(df.Type=="Real") & (df.Phase==i+1)]["Normalized mean pixel intensity"].values[0], df[(df.Type=="Real") & (df.Phase==i+2)]["Normalized mean pixel intensity"].values[0]], 
             linewidth=1, color=palette[i+1], zorder=0)
    plt.plot([i+1, i+2], [df[(df.Type=="Synthetic") & (df.Phase==i+1)]["Normalized mean pixel intensity"].values[0], df[(df.Type=="Synthetic") & (df.Phase==i+2)]["Normalized mean pixel intensity"].values[0]], 
             linewidth=1, color=palette[i+1], zorder=0)
    plt.plot([i+1, i+2], [df_pre_["Normalized mean pixel intensity"].unique()[0], df_pre_["Normalized mean pixel intensity"].unique()[0]], 
             linewidth=1, color="0.55", zorder=0)
    
handles, lables = ax.get_legend_handles_labels()
for h in handles[0:5]+handles[11:]:
    h.set_markersize(10)
lgnd = plt.legend(handles, lables, bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)

ax.set_xticks(list(range(1, 5)))
ax.set_xticklabels(['1','2','3','4'])

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel("Phase", fontsize=12)
plt.ylabel("Normalized mean pixel intensity", fontsize=12)
plt.show()
# plt.savefig("plots/20240306_gs0_cgs1p0_200steps_scatter_real_synthetic_NORMALIZED_INTENSITY_all_phases_accumulated_test_set_downsize_real_mask.png", bbox_inches='tight', dpi=300)